In [ ]:
import pandas as pd
from src.utils.data_utils import get_demand_df, update_demand_dic, load_matrix_df, get_demand_matrix_df, load_df
from src.utils.helper_utils import get_penalty_list, get_values_not_in_second_list
from src.utils.route_utils import sort_nodes_by_distance

In [ ]:
demand_df = get_demand_df(
    today_path='../data/orders/03-03-2025-PO.csv'
    )
demand_df.shape

In [ ]:
demand_dict = update_demand_dic(demand_df)

In [ ]:
distance_mat = load_matrix_df('../data/master/osrm_distance_matrix.csv')

In [ ]:
demand_mat_df = get_demand_matrix_df(distance_mat,demand_df, 0)

In [ ]:
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
penalty_lis = get_penalty_list(demand_dict, DISTANCE_BASE_PENALTY, 6, today)

In [ ]:
demand_sort_nodes = sort_nodes_by_distance(demand_mat_df)

In [ ]:
nodes_to_visit = demand_sort_nodes[:300]

In [ ]:
from src.data_model.vrp_data_model import VRPDataModel

In [ ]:
dataClass = VRPDataModel(
    full_matrix=demand_mat_df,
    nodes_to_visit=nodes_to_visit,
    demand_dict=demand_dict,
    penalty_list=penalty_lis,
    max_visits=[8]*8
)

In [ ]:
data=dataClass.get_data()

In [ ]:
len(data['distance_matrix'])

In [ ]:
from src.solver.vrp_solver import VRPSolver

In [ ]:
solver = VRPSolver()

In [ ]:
visited_nodes, route_dict = solver.solve_day(data=data, day=1)

In [1]:
from src.controller.controller import VRPController

# === File paths ===
demand_path = '../data/orders/03-03-2025-PO.csv'
matrix_path = '../data/master/osrm_distance_matrix.csv'
gps_path = '../data/master/master_gps.csv'
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
total_days = 6

# === Initialize Controller ===
controller = VRPController()

controller.load_inputs(
    demand_path=demand_path,
    matrix_path=matrix_path,
    gps_path=gps_path,
    base_penalty=DISTANCE_BASE_PENALTY,
    total_days=total_days,
    today=today
)

# === Predefined route assignment ===
vehicle_routes = {
    0: 1,  # vehicle_id 0 uses route_id 1 from predefined routes
    1: 2
}

# === Configure Vehicles ===
controller.configure_vehicles(
    num_vehicles=8,
    max_visits=[12] * 8,
    max_distance=[400] * 8,
    vehicle_routes=vehicle_routes
)

# === Solve VRP ===
visited_nodes, routes = controller.solve_single_day()

# === Print results ===
print("Visited Nodes (Predefined + Solved):", visited_nodes)
print("Routes by Vehicle ID:")
for vid, route in routes.items():
    print(f"Vehicle {vid}: {route}")

shop codes len:32
create_data_model called with:
max_distance: [400]
max_visits: [12]
Distance Matrix Shape: (651, 651)
Length of nodes_to_visit: 32

Day 2 Routes:
Route for vehicle 0:
 0 -> 1331 -> SINL -> SIWD -> SIP2 -> 1781 -> SFAB -> 1466 -> 1117 -> 1847 -> 1370 -> 1809 -> 1787 -> 0
Distance of route: 239 km (Max: 400)
Stops visited: 12/12

Max route distance for Day 2: 239 km

{0: {'route_nodes': ['0', '1331', 'SINL', 'SIWD', 'SIP2', '1781', 'SFAB', '1466', '1117', '1847', '1370', '1809', '1787', '0'], 'route_distance': 239, 'max_distance_limit': 400, 'within_limit': True, 'num_visits': 12, 'max_visits_limit': 12}}
shop codes len:22
create_data_model called with:
max_distance: [400]
max_visits: [12]
Distance Matrix Shape: (651, 651)
Length of nodes_to_visit: 22

Day 2 Routes:
Route for vehicle 0:
 0 -> 1170 -> SKKM -> 1460 -> 1113 -> 122 -> 1159 -> SYMT -> 1659 -> 1481 -> 1124 -> 1350 -> 1770 -> 0
Distance of route: 263 km (Max: 400)
Stops visited: 12/12

Max route distance for D

In [3]:
from src.utils.visualization import visualize_routes_per_vehicle, load_route_cache, print_route_summary, save_route_details_to_csv

In [4]:
load_route_cache()

No cache file found at ../data/csv/route_cache.csv, starting fresh


In [5]:
maps = visualize_routes_per_vehicle(
    master_df=controller.master_gps_df,
    route_dict=routes,
    day=0,
    use_distance=True  # or False if using time
)

No valid path found between SINL and SIWD, skipping.
No valid path found between SIWD and SIP2, skipping.
No valid path found between 1466 and 1117, skipping.
No valid path found between 1460 and 1113, skipping.
No valid path found between 1659 and 1481, skipping.
No valid path found between 1481 and 1124, skipping.
No valid path found between SLMN and SCBI, skipping.
No valid path found between 14010 and 110, skipping.
No valid path found between 1084 and SGKC, skipping.
No valid path found between 14013 and 171, skipping.
No valid path found between SCMY and 1137, skipping.
No valid path found between 14015 and SGKR, skipping.
No valid path found between SGKR and 3, skipping.
Saved 91 cached routes to ../data/csv/route_cache.csv


In [2]:
from src.controller.controller import VRPController, RouteMapManager

# === File paths ===
demand_path = '../data/orders/03-03-2025-PO.csv'
matrix_path = '../data/master/osrm_distance_matrix.csv'
gps_path = '../data/master/master_gps.csv'
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
total_days = 6

# Setup
controller = VRPController()
controller.load_inputs(demand_path, matrix_path, gps_path, DISTANCE_BASE_PENALTY, total_days, today)
controller.configure_vehicles(8, [12]*8, [200]*8)

# Solve
visited_nodes, route_dict = controller.solve_single_day()

# Visualize & Save
visualizer = RouteMapManager(controller.master_gps_df, controller.demand_df)
visualizer.generate_and_save_maps(route_dict, day=0)
visualizer.summarize_and_save(route_dict, day=0)


create_data_model called with:
max_distance: [200, 200, 200, 200, 200, 200, 200, 200]
max_visits: [12, 12, 12, 12, 12, 12, 12, 12]
Distance Matrix Shape: (651, 651)
Length of nodes_to_visit: 457

Day 2 Routes:
Route for vehicle 0:
 0 -> 1301 -> 1446 -> 1859 -> 1661 -> 1105 -> 1826 -> 1902 -> 1017 -> 5 -> 1433 -> 1364 -> SGTY -> 0
Distance of route: 31 km (Max: 200)
Stops visited: 12/12

Route for vehicle 1:
 0 -> 1555 -> 1545 -> 1145 -> 14003 -> 1303 -> SCUP -> SCSL -> 1759 -> SCDR -> 1746 -> 1776 -> 1896 -> 0
Distance of route: 31 km (Max: 200)
Stops visited: 12/12

Route for vehicle 2:
 0 -> 22 -> SCTG -> 14012 -> SCBT -> 116 -> SCPK -> SCVJ -> SCHD -> SCTL -> SCAK -> SCKX -> SGSM -> 0
Distance of route: 42 km (Max: 200)
Stops visited: 12/12

Route for vehicle 3:
 0 -> 1548 -> 1549 -> SCTH -> SLMN -> SCBI -> SCPT -> 110 -> 14010 -> 1678 -> 1355 -> 1338 -> SGMQ -> 0
Distance of route: 47 km (Max: 200)
Stops visited: 12/12

Route for vehicle 4:
 0 -> SGEL -> SGK5 -> SGMP -> SGGP -> 24 